# 1. Data Load

In [1]:
# load required packages
import pandas as pd
import matplotlib

## 1.1 Charging Stations

In [115]:
# load charging station data from opendata portal of Rhein-Kreis-Neuss found via govdata.de
# skip bad lines
stations_raw = pd.read_csv("https://opendata.rhein-kreis-neuss.de/explore/dataset/rhein-kreis-neuss-ladesaulen-in-deutschland/download/?format=csv",
                       on_bad_lines = "skip",
                       sep = ";")

## 1.2 Registered Vehicles

In [116]:
# load vehicle registration data from opendata portal from Kraftfahrtbundesamt
vehicles_raw = pd.read_excel("https://www.kba.de/SharedDocs/Downloads/DE/Statistik/Fahrzeuge/FZ3/fz3_2022.xlsx?__blob=publicationFile&v=3", sheet_name = "FZ 3.1")

# 2. Data Cleaning

## 2.1Charging Stations

In [117]:
# copy raw data to working df
stations = stations_raw.copy()

# replace column names with english names
stations.columns = ["owner",
                    "count",
                    "connected_load",
                    "connector_1",
                    "connector_2",
                    "connector_3",
                    "connector_4",
                    "p1",
                    "p2",
                    "p3",
                    "p4",
                    "district",
                    "municipality",
                    "zip",
                    "street_name",
                    "street_number",
                    "additional_address",
                    "installation_date",
                    "coordinates",
                    "type"]

# split coordinates into separate columns
stations[["latitude", "longitude"]] = stations.coordinates.str.split(",", expand = True)

# remove coordinates column
stations = stations.drop("coordinates", axis = 1)

## 2.2 Registered Vehicles

In [137]:
# copy raw data to working df
vehicles = vehicles_raw.copy()

# remove empty first column
vehicles.drop(vehicles.columns[0], axis = 1, inplace = True)

# drop first rows containing no usable data
vehicles.drop(vehicles.index[0:8], axis = 0, inplace = True)

# drop last rows containing no usable data
vehicles.drop([11209, 11210, 11211, 11212, 11213, 11214, 11215 ,11216, 11217, 11218, 11219],
              axis = "index",
              inplace = True)

# reset index
vehicles.reset_index(drop = True, inplace = True)

# assign proper column names
vehicles.columns = ["state",
                    "district_raw",
                    "municipality_raw",
                    "motorcycles",
                    "cars_total",
                    "cars_business",
                    "trucks",
                    "tractors_total",
                    "tractors_agri_forest",
                    "other_vehicles_buses",
                    "vehicles_total",
                    "trailers"]


# fill NaN values
# replace in these columns
for column in ["state", "district_raw"]:
    # loop through all the rows
    for row in range(0,len(vehicles)):
        # check if value is NaN
        if vehicles.iloc[row][column] != vehicles.iloc[row][column]:
            # replace with value from previous row if NaN
            vehicles.iloc[row][column] = vehicles.iloc[row-1][column]

# remove summary rows
# create list to store row indexes to be dropped
drop = []

# search in municipality column
column = "municipality_raw"

# loop through all the rows
for row in range(0, len(vehicles)):
    # check if row is summary row
    if vehicles.iloc[row][column] == "ZUSAMMEN":
        # append to list to be dropped later
        drop.append(row)

# search in state column
column = "state"
# loop through all the rows
for row in range(0, len(vehicles)):
    # check if row is summary row
    if "INSGESAMT" in vehicles.iloc[row][column]:
        # append to list to be dropped later
        drop.append(row)

# iterate through droppable list and execute drop
for row in drop:
    vehicles.drop([row], axis = "index", inplace = True)

# reset index
vehicles.reset_index(drop = True, inplace = True)

# extract district id
vehicles["district_id"] = vehicles["district_raw"].str[-8:].str[2:7]

#extract district name
vehicles["district"] = vehicles["district_raw"].str[:-8]

# extract zip code from municipality
vehicles["zip"] = vehicles["municipality_raw"].str[0:5]

#extract name code from municipality
vehicles["municipality"] = vehicles["municipality_raw"].str[7:]

# drop municipality_raw and district_raw
vehicles.drop(["municipality_raw", "district_raw"], axis = 1, inplace = True)

# reorder columns
vehicles = vehicles[["state",
                     "district_id",
                     "district",
                     "zip",
                     "municipality",
                     "motorcycles",
                     "cars_total",
                     "cars_business",
                     "trucks",
                     "tractors_total",
                     "tractors_agri_forest",
                     "other_vehicles_buses",
                     "vehicles_total",
                     "trailers"]]

# replace "." & "-" value with 0
# execute replace in following columns
for column in ["motorcycles",
               "cars_total",
                "cars_business",
                "trucks",
                "tractors_total",
                "tractors_agri_forest",
                "other_vehicles_buses",
                "vehicles_total",
                "trailers"]:
    vehicles.loc[vehicles[column] == ".", column] = 0
    vehicles.loc[vehicles[column] == "-", column] = 0

# assign correct column types
vehicles = vehicles.astype({"state": str,
                            "district_id": str,
                            "district": str,
                            "zip": str,
                            "municipality": str,
                            "motorcycles": int,
                            "cars_total": int,
                            "cars_business": int,
                            "trucks": int,
                            "tractors_total": int,
                            "tractors_agri_forest": int,
                            "other_vehicles_buses": int,
                            "vehicles_total": int,
                            "trailers": int})

# show data
vehicles

,state,district_id,district,zip,municipality,motorcycles,cars_total,cars_business,trucks,tractors_total,tractors_agri_forest,other_vehicles_buses,vehicles_total,trailers
0,BADEN-WÜRTTEMBERG,08111,"STUTTGART,STADT",70173,"STUTTGART,LANDESHAUPTSTADT",26692,302792,76738,17458,2841,1726,2341,352124,19214
1,BADEN-WÜRTTEMBERG,08115,BOEBLINGEN,71134,AIDLINGEN,722,6196,311,232,287,192,16,7453,1076
2,BADEN-WÜRTTEMBERG,08115,BOEBLINGEN,71155,ALTDORF,333,2952,117,0,151,91,0,3557,388
3,BADEN-WÜRTTEMBERG,08115,BOEBLINGEN,71032,"BOEBLINGEN,ST.",2565,31420,5515,1240,291,191,263,35779,2304
4,BADEN-WÜRTTEMBERG,08115,BOEBLINGEN,71149,BONDORF,449,3750,184,238,324,213,14,4775,699
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10783,THÜRINGEN,16077,ALTENBURGER LAND,04617,STARKENBERG,142,1304,68,142,99,85,20,1707,396
10784,THÜRINGEN,16077,ALTENBURGER LAND,04626,THONHAUSEN,0,354,30,88,60,56,0,556,160
10785,THÜRINGEN,16077,ALTENBURGER LAND,04617,TREBEN,99,706,34,84,0,0,0,923,207
10786,THÜRINGEN,16077,ALTENBURGER LAND,04626,VOLLMERSHAIN,32,211,4,0,16,16,0,276,68


# 3. Data Merge

# 4. KPI Computation

# 5. Visualization

## 5.1 Visualization Preparation

## 5.2 Visualization Creation